In [ ]:
using Plots
using QuadGK
using NLopt
#using SpecialFunctions
using NLsolve
using Statistics
#include("../../scaling_CGS.jl")
include("../../code_to_real.jl")

In [ ]:
#-------- From Dahldorf et al. 2014

c_code = 1.0
B_code = 0.04 #* sqrt(4*pi) # Multiply with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units
p_code = 4.5e-4
rho_code = 1.0
debye_code = 0.015 #/ sqrt(4*pi) # divide with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units



c_u = code_units(c_code, B_code, p_code, rho_code, debye_code)

number_density_real = 1e9 # Guess 


r_u = known_real_units(number_density_real)
eps_0_scaling = 1e2
charge_scaling = 1e0
electron_proton_mass_ratio = 100

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


scaling = find_real_units_CGS(c_u, r_u, r_s)
print_basic_info(scaling)
print_all_CGS(scaling)

In [ ]:
3.9968e-02 / 1.1283e-02

In [ ]:
#-------- From Dahldorf et al. 2014

c_code = 1.0
B_code = 0.04 #* sqrt(4*pi) # Multiply with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units
p_code = 4.5e-4
rho_code = 1.0
debye_code = 0.015 #/ sqrt(4*pi) # divide with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units



c_u = code_units(c_code, B_code, p_code, rho_code, debye_code)

number_density_real = 1e9 # Guess 


r_u = known_real_units(number_density_real)
eps_0_scaling = 1e2
charge_scaling = 1e0
electron_proton_mass_ratio = 25.0

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


scaling = find_real_units_HL(c_u, r_u, r_s)


ds = 0.1
ppc = 128
set_macro_particle_weights(scaling, ds, ppc)
print_basic_info(scaling)
print_all_HL(scaling)

In [ ]:
v = scaling.v_thermal_e_code
c = 1.0

v_squared = v^2
gamma = 1.0 / sqrt(1.0 - v_squared / c^2)
(gamma - 1. ) * 100


E_clas = 0.5 * scaling.m_e_code * v^2
E_rel = (gamma - 1.0) * scaling.m_e_code * c^2

println("E_clas = $E_clas")
println("E_rel = $E_rel")
println("E_rel / E_clas = $(E_rel / E_clas)")


In [ ]:
gamma - 1 / gamma

In [ ]:
#-------- From Dahldorf et al. 2014

c_code = 1.0
B_code = 0.04 #* sqrt(4*pi) # Multiply with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units
p_code = 4.5e-4
rho_code = 1.0
debye_code = 0.015 #/ sqrt(4*pi) # divide with sqrt(4*pi) to get it in CGS - they have in Heaviside-Lorentz units



c_u = code_units(c_code, B_code, p_code, rho_code, debye_code)

number_density_real = 1e9 # Guess 


r_u = known_real_units(number_density_real)
eps_0_scaling = 1e2
charge_scaling = 1e0
electron_proton_mass_ratio = 100.0

r_s = ratios_and_scaling(electron_proton_mass_ratio, eps_0_scaling, charge_scaling)


scaling = find_real_units_HL(c_u, r_u, r_s)
print_basic_info(scaling)
print_all_HL(scaling)

In [ ]:
scaling.e_code / scaling.m_e_code

In [ ]:
5.001 * sqrt(4 * pi )

In [ ]:
ds = 32 / (32 * 16)
per_cell = 32

set_macro_particle_weights(scaling, ds, per_cell);

print_all_HL(scaling)


In [ ]:
cf = sqrt(B_code^2 + 2 * p_code / rho_code) # fast magnetosonic speed

cf * 300 

In [ ]:
lambda = 32.0
k = 2 * pi / lambda

Lx = 160/3
Ly = 40

n_patch_x = 16
n_cells_x = 32

n_patch_y = 12
n_cells_y = 32


dx = Lx / (n_patch_x * n_cells_x)
dy = Ly / (n_patch_y * n_cells_y)

nx = n_patch_x * n_cells_x
ny = n_patch_y * n_cells_y
println("dx = $dx")
println("dx = $dy")
println("dx / 2 = $(dx / 2)")

println("nx = $nx")
println("ny = $ny")


per_cell = 32

In [ ]:
0.10416666666666667 / 2

In [ ]:
0.10416666666666667 / scaling.debye_len_code

In [ ]:
#phi0 = atan(4/3.)
phi0 = 0.

Lx = lambda # / cos(phi0)
Ly = lambda # / sin(phi0)
println("phi0 = $phi0")
println("Lx = $Lx")
println("Ly = $Ly")



n_patch_x = 16
n_cells_x = 32

n_patch_y = 16
n_cells_y = 32


dx = Lx / (n_patch_x * n_cells_x)
dy = Ly / (n_patch_y * n_cells_y)

nx = n_patch_x * n_cells_x
ny = n_patch_y * n_cells_y
println("dx = $dx")
println("dx = $dy")

println(" ds / 2 = $(dx / 2)")

println("nx = $nx")
println("ny = $ny")


per_cell = 32


In [ ]:
dx / debye_code

In [ ]:
gamma = 1.4

B_code_new  = B_code# * sqrt(4 * pi )

c_s = sqrt( 2 * p_code / rho_code)
v_a = B_code_new / sqrt(4 * pi *  rho_code)


sqrt(v_a^2 + c_s^2)

In [ ]:
c_f = sqrt( (B_code_new^2/ (4 *pi) + gamma * p_code) / rho_code)

In [ ]:
32 / c_f

In [ ]:
n = 1000
x = range(-16,16,n)

x_static = ones(n) 

delta = 0.1
lambda = 32. 
k = 2 * pi / lambda



B_y_arr = B_code_new .* ( 1. .+ delta .* sin.(k * x))
B_pressure = B_y_arr.^2 ./ (8 * pi)


delta_p = 0.1^2 * B_code_new^2 / (8 * pi ) * rho_code / (p_code * gamma) #1 / sqrt(9 * pi )  #delta * (B_code_new^2 / (8 * pi) ) 

p_code_arr = p_code .* ( 1. .+ delta_p .* sin.(k * x))

ratio = p_code_arr ./ B_pressure 

ratio_static =  p_code / (B_code_new^2 / (8 * pi) )   .* x_static


p1 = plot(x, p_code_arr, label="p_code")
p1 = plot!(x, B_pressure, label="B_y")

p2 = plot(x, ratio, label="ratio")
p2 = plot!(x, ratio_static, label="ratio_static")

plot(p1, p2, layout=(1,2), size=(1000,400))


In [ ]:
sin(7*pi/8)

In [ ]:
delta = 0.1
lambda = 32.
k = 2 * pi / lambda


x = -14


B_val = scaling.B_flux_code * (1.0 + delta * sin(k * x ))
grad_B = scaling.B_flux_code * delta * k * cos( k * x)
curl_grad_B = - scaling.B_flux_code * (1.0 + delta * sin(k * x )) * grad_B



In [ ]:

v_drift_elec = 4 / (3 * pi ) * (- scaling.e_code) * scaling.pressure_code / (scaling.mass_density_code * scaling.m_e_code^2 * scaling.rho_n_ratio) * curl_grad_B / B_val^3

#v_drift_prot = 4 / (3 * pi ) * (+ scaling.e_code) * scaling.pressure_code / (scaling.mass_density_code * scaling.m_p_code^2 * scaling.rho_n_ratio) * curl_grad_B / scaling.B_flux_code^3

In [ ]:
0.12 / scaling.B_flux_code

In [ ]:
1 /  3e10^2